In [1]:
import ipykernel
ipykernel._version

<module 'ipykernel._version' from 'c:\\ProgramData\\anaconda3\\envs\\ia\\lib\\site-packages\\ipykernel\\_version.py'>

In [2]:
import time
import cv2
import torch as t
import albumentations as a
import numpy as np
from torch.utils.data import Dataset
from torchvision import transforms
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw

c:\ProgramData\anaconda3\envs\ia\lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.13 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


In [3]:
total_time = 0
file_path = ["C:/Users/yhd06/OneDrive/Desktop/iwaz/pothole_data/pothole_dataset/labels/0007648.jpg"]
label_path = ["C:/Users/yhd06/OneDrive/Desktop/iwaz/pothole_data/pothole_dataset/labels/0007648.txt"]

In [4]:
class AlbumentationsDataset(Dataset):
    def __init__(self, file_paths, labels_paths, transform=None):
        self.file_paths = file_paths
        self.labels_paths = labels_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label_path = self.labels_paths[idx]
        
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        labels = self._load_labels(label_path, image.shape[1], image.shape[0])

        start_t = time.time()
        if self.transform:
            augmented = self.transform(image=image, bboxes=labels['bboxes'], category_ids=labels['category_ids'])
            image = augmented['image']
            labels = {'bboxes': augmented['bboxes'], 'category_ids': augmented['category_ids']}
        total_time = (time.time() - start_t)

        return image, labels, total_time
    
    def _load_labels(self, labels_path, img_width, img_height):
        bboxes = []
        category_ids = []
        with open(labels_path, 'r') as file:
            for line in file.readlines():
                parts = line.strip().split()
                category_ids.append(int(parts[0]))
                x_center, y_center, width, height = map(float, parts[1:])
                x_center /= img_width
                y_center /= img_height
                width /= img_width
                height /= img_height

                bboxes.append([
                    x_center - width / 2,
                    y_center - height / 2,
                    x_center + width / 2,
                    y_center + height / 2
                ])
        
        return {'bboxes': bboxes, 'category_ids': category_ids}

In [5]:
albumentations_transform = a.Compose([
    a.HorizontalFlip(p=0.5),
    a.RandomBrightnessContrast(p=0.2),
    a.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5),
], bbox_params=a.BboxParams(format='yolo', label_fields=['category_ids']))

In [6]:
albumentations_dataset = AlbumentationsDataset(
    file_paths = file_path,
    labels_paths = label_path,
    transform=albumentations_transform,
)

In [7]:
for i in range(20):
  sample, bbox, transform_time = albumentations_dataset[0]
  total_time += transform_time

print("albumentations time/sample: {} ms".format(total_time*10))

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
def draw_boxes(image_path, labels_path):

    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    with open(labels_path, 'r') as file:
        lines = file.readlines()

    width, height = image.size

    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1]) * width
        y_center = float(parts[2]) * height
        bbox_width = float(parts[3]) * width
        bbox_height = float(parts[4]) * height
        
        left = x_center - bbox_width / 2
        top = y_center - bbox_height / 2
        right = x_center + bbox_width / 2
        bottom = y_center + bbox_height / 2

        draw.rectangle([left, top, right, bottom], outline="red", width=2)

    plt.imshow(np.array(image))
    plt.axis('off')
    plt.show()

file_path = "C:\\Users\\yhd06\\OneDrive\\Desktop\\iwaz\\pothole_data\\output_potehole_dataset\\images\\0011245_aug_2.jpg"
label_path = "C:\\Users\\yhd06\\OneDrive\\Desktop\\iwaz\\pothole_data\\output_potehole_dataset\\images\\0011245_aug_2.txt"
draw_boxes(file_path, label_path)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\yhd06\\OneDrive\\Desktop\\iwaz\\pothole_data\\output_potehole_dataset\\images\\0011245_aug_2.txt'